In [1]:
import torch
import torch.nn.functional as F
torch.cuda.is_available()

True

# Import Text Document of Names

In [2]:
words = open('names.txt', 'r', encoding="utf-8").read().splitlines()

In [3]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

### Create training set of bigrams

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0 #denotes the beginning and ending of words
itos = {i:s for s,i in stoi.items()}

In [5]:
stoi

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '.': 0}

In [6]:
xs, ys = [], [] #x = input, y = target

for w in words:
    chars = ['.'] + list (w) + ['.']
    for char1, char2 in zip(chars, chars[1:]):
        index1 = stoi[char1]
        index2 = stoi[char2]
        xs.append(index1)
        ys.append(index2)

        
#conver to tensors
xs = torch.tensor(xs)
ys = torch.tensor(ys)        
num = xs.nelement()

In [7]:
print(f'Number of examples: {num}')

Number of examples: 228146


### Initialize Generator for Reproducibility

In [8]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27), generator=g, requires_grad=True) #generate 27 x 27 vectors of random weights

## Optimizaiton

In [9]:
for k in range(100): #epochs
    #forward pass
    x_encoding = F.one_hot(xs, num_classes=27).float() #for each character (index), create a 1d array of length 27 
                                                       #(26 letters + special start/end char) where the chars index is 1 while all else is 0
    logits = (x_encoding @ W) #  log counts #matrix multiplication sign in pytorch, the resulting 5,27 tensor tells us the firing rate of each neuron given these 5 examples
    counts = logits.exp() # better approximation of counts because always positive
    probs = counts / counts.sum(1, keepdim=True) # normalize to approximate individual probabilities for each letter in the vector. counts into probs is called softmax
    loss = -probs[torch.arange(num), ys].log().mean() # negative log likelihood
    print(loss.item())
    
    #backward pass
    W.grad = None # set the zero to gradient
    loss.backward() # fills in intermediary gradients all the way back to W
    
    #update weights
    W.data += -50 * W.grad   # each element here describes the effect on the loss function in the form of 27x27 tensor
                        # essentially setting the learning rate to -.1 (which is negative to minimize loss function)

3.758953809738159
3.3710803985595703
3.1540329456329346
3.0203664302825928
2.9277069568634033
2.8604001998901367
2.809727430343628
2.7701010704040527
2.738072633743286
2.711496114730835
2.6890032291412354
2.6696887016296387
2.6529300212860107
2.638277053833008
2.6253879070281982
2.613990545272827
2.60386323928833
2.5948219299316406
2.586712121963501
2.57940411567688
2.572789192199707
2.5667765140533447
2.5612881183624268
2.5562589168548584
2.551633834838867
2.547366142272949
2.5434155464172363
2.5397486686706543
2.536336660385132
2.5331544876098633
2.5301806926727295
2.5273966789245605
2.5247862339019775
2.522334575653076
2.520029067993164
2.5178580284118652
2.515810489654541
2.513878345489502
2.512052059173584
2.510324001312256
2.5086872577667236
2.5071346759796143
2.5056614875793457
2.504261016845703
2.502929210662842
2.5016613006591797
2.5004522800445557
2.4992990493774414
2.498197317123413
2.497144937515259
2.496137857437134
2.495173692703247
2.4942495822906494
2.493363380432129
2.

In [26]:
g = torch.Generator().manual_seed(2147483647)

for i in range(50):
    output = []
    index = 0
    while True:
        x_encoding = F.one_hot(torch.tensor([index]), num_classes=27).float()
        logits = x_encoding @ W # predict log counts
        counts = logits.exp()
        probs = counts / counts.sum(1, keepdims=True)
        #------------------------
        index = torch.multinomial(probs, num_samples=1,replacement=True, generator=g).item()
        output.append(itos[index])
        if index == 0:
            break
    print(''.join(output))

mor.
axwaninaymoryles.
kondmaisah.
anchshizarie.
odaren.
iaddash.
h.
jionatien.
egwver.
ga.
t.
a.
jayn.
ilemannariaenien.
ad.
f.
akiinela.
trttanakerorudayaaxetona.
lamoyonutonadengin.
torrederahmokallovwiprasskh.
a.
wai.
kn.
jaieendenelfff.
kianu.
eryly.
zeeieil.
kayxtrglynenn.
r.
t.
rja.
kronanuroliojanidames.
dikie.
s.
elarieiavelvalllaish.
eythann.
janianaeri.
boma.
poree.
l.
adhanoena.
naisaryan.
sialaitalyee.
cena.
lisarr.
bizyngor.
eve.
blahish.
tovih.
al.
